In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os
import numpy as np
from prosit_t.models import PrositTransformerV2
from dlomix.losses import masked_spectral_distance

2023-10-06 09:45:34.061992: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 09:45:34.975659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tf.config.run_functions_eagerly(True)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
sequences_data = [
    ["S","E","Q"],
    ["S","E"],
    ["S","E","Q", "W"],
    ["E", "Q"],
    ["S","E","Q", "W", "E", "N", "C", "E"],
    ["S","E","Q", "W", "E", "N", "C", "E", "T", "Y"],
]

labels_data = []
for i in sequences_data:
    labels_data.append(list(range(len(i)-1))*6)

charge_data = [
    [0,1,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [0,1,0,0,0,0]
]
ce_data = [
    [0.215],
    [0.24],
    [0.287],
    [0.213],
    [0.215],
    [0.234]
]
sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.string)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.float64)
charge_ragged = tf.ragged.constant(charge_data, dtype=tf.float32)
ce_ragged = tf.ragged.constant(ce_data, dtype=tf.float64)

2023-10-06 09:45:37.727590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43640 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [5]:
def ragged_to_dense(x,y):
    return x, y

def merge_tuples(item1, item2):
    return ({
        "sequence": item1[0],
        "collision_energy": item2[1],
        "precursor_charge": item2[0],
    }, item1[-1])

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(
            (
                sequences_ragged,
                labels_ragged,
            )
        ).map(ragged_to_dense).padded_batch(2).unbatch()

(None,) (None,)


/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [7]:
dataset_meta = tf.data.Dataset.from_tensor_slices(
    (
        charge_ragged,
        ce_ragged
    )).map(ragged_to_dense)

(None,) (None,)


In [8]:
full = tf.data.Dataset.zip(dataset, dataset_meta).map(merge_tuples).batch(2).prefetch(tf.data.AUTOTUNE)

In [10]:
from prosit_t.models.variable_seq_length_models import TestModelDrop
from prosit_t.models.variable_seq_length_models import TestModelPooling
import keras.backend as K

In [11]:
test = TestModelDrop(embedding_output_dim=64,num_heads=16,num_transformers=3,dense_dim_factor=4)

In [12]:
test.compile(loss=masked_spectral_distance, optimizer='adam')

In [13]:
test.fit(full, epochs=10)

Epoch 1/10


2023-10-06 09:45:39.028473: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-06 09:45:40.528534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801


3/3 [==============================] - 3s 189ms/step - loss: 0.8874
Epoch 2/10
3/3 [==============================] - 0s 158ms/step - loss: 0.6140
Epoch 3/10
3/3 [==============================] - 0s 160ms/step - loss: 0.5821
Epoch 4/10
3/3 [==============================] - 0s 160ms/step - loss: 0.5913
Epoch 5/10
3/3 [==============================] - 0s 158ms/step - loss: 0.5754
Epoch 6/10
3/3 [==============================] - 0s 158ms/step - loss: 0.5663
Epoch 7/10
3/3 [==============================] - 0s 160ms/step - loss: 0.5524
Epoch 8/10
3/3 [==============================] - 0s 158ms/step - loss: 0.5537
Epoch 9/10
3/3 [==============================] - 0s 157ms/step - loss: 0.5274
Epoch 10/10
3/3 [==============================] - 0s 157ms/step - loss: 0.5325


In [103]:
import plotly.express as px

fig = px.line(x=list(range(1,11)), y=test.history.history["loss"], title='TimeDistributed Dense Layer')
fig.update_xaxes(title='Epoch')
fig.update_yaxes(title='Loss')

In [104]:
test.summary()

Model: "test_model_decoder_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    multiple                  0 (unused)
                                                                 
 flatten_17 (Flatten)        multiple                  0         
                                                                 
 time_distributed_17 (TimeD  multiple                  390       
 istributed)                                                     
                                                                 
 string_lookup_17 (StringLo  multiple                  0         
 okup)                                                           
                                                                 
 positional_embedding_17 (P  multiple                  1408      
 ositionalEmbedding)                                             
                                              